In [3]:
import pandas as pd
import numpy as np
#--------------Visualization--------------#
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(rc={'figure.figsize': [12, 7]}, font_scale=1.3)
import plotly.express as px
import plotly.graph_objs as go
import plotly

from plotly import tools
import matplotlib.ticker as mtick

In [536]:
# Load the 'basic data' sheet
basic_data = pd.read_excel(r'C:\Users\salma.elsayed\Desktop\basic data\17 NOV\\17 Nov. 2024.xlsx')
all_ = pd.read_excel(r'C:\Users\salma.elsayed\Desktop\basic data\17 NOV\position correct.xlsx',sheet_name='all (2)')
joblist = pd.read_excel(r'C:\Users\salma.elsayed\Desktop\basic data\17 NOV\position correct.xlsx',sheet_name='joblist')
action_list = pd.read_excel(r'C:\Users\salma.elsayed\Desktop\basic data\17 NOV\\17 Nov. 2024 - Action List.xlsx')

KeyboardInterrupt: 

In [360]:
def Data_Overview (basic_data):
    print(f"- Dataset Shape: {basic_data.shape}")
    
    print("--"*20)
    
    print(f'- Daplication data: {basic_data.duplicated().sum()}')
    
    print("--"*20)
    
    summary = pd.DataFrame(basic_data.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = basic_data.isnull().sum().values 
    summary['Missing %'] = round(basic_data.isnull().mean()*100,2).values    
    summary['Uniques'] = basic_data.nunique().values
    return summary.style.background_gradient(cmap ='RdBu_r')

Data_Overview(basic_data)

- Dataset Shape: (24646, 35)
----------------------------------------
- Daplication data: 0
----------------------------------------


,Name,dtypes,Missing,Missing %,Uniques
0,Company,object,0,0.000000,22
1,Personnel No.,int64,0,0.000000,24631
2,Previous SAP ID,float64,12833,52.070000,11377
3,Employee Name,object,0,0.000000,24353
4,Employee Arabic name,object,7,0.030000,24049
5,Position,int64,0,0.000000,17097
6,Position.1,object,0,0.000000,1548
7,Concatenate - Position,object,0,0.000000,2383
8,Service Type,float64,24646,100.000000,0
9,Job,object,12646,51.310000,964


## Data cleaning for basic data

In [ ]:
inx=basic_data[(basic_data['Status']=='Withdrawn' ) & (basic_data ['Reason for Action']!= 'Transfer to sister company')].index
inx2=basic_data[(basic_data['Status']=='Inactive' ) & (basic_data ['Action']!= 'Unpaid Leave') ].index
inx3=basic_data[basic_data['Reason for Action']=='Management approval'].index

all_indexes_to_drop = inx.union(inx2).union(inx3)

basic_data.drop(all_indexes_to_drop, axis=0, inplace=True)

basic_data.reset_index(drop=True, inplace=True)

In [ ]:
# Define a dictionary with incorrect job titles as keys and corrected job titles as values
job_title_corrections = {
    "project controls director": "projects control director",##الي هيبقى موجود
    "senior pre- fabrication manager": "senior pre-fabrication manager",
    "tendering&contracts support senior engin": "senior tendering&Contracts support engin",
    "senior commerical engineer": "senior commercial engineer",
    "chief surveyor - bridges": "chief surveyor",
    "laboratory technician ( concrete )": "lab technician",
    "laboratory technician ( roads )": "lab Technician",
    "corporate&business development director": "business & corporate development director",
    "project controls senior director": "projects control senior director"
}
# Apply the corrections to the Position.1 column in both dataframes
basic_data['Position.1'] = basic_data['Position.1'].str.lower().map(job_title_corrections).fillna(basic_data['Position.1'])
action_list['New position'] = action_list['New position'].str.lower().map(job_title_corrections).fillna(action_list['New position'])

action_list.drop_duplicates(inplace = True)
action_list.reset_index(drop=True, inplace=True)

In [ ]:
basic_data['Company']=basic_data['Company'].str.lower().str.strip()
basic_data['Position.1']= basic_data['Position.1'].str.lower().str.strip()
basic_data['Concatenate - Position_2'] = basic_data['Company'] + basic_data['Position.1']
joblist['Position - Job List']= joblist['Position - Job List'].str.lower().str.strip()
joblist['Company']=joblist['Company'].str.lower().str.strip()
joblist['Concatenate - Position_2'] = joblist['Company'] + joblist['Position - Job List']

In [ ]:
# تحويل القيم بعد المقارنة إلى أحرف صغيرة (lower)
basic_data['Company'] = np.where(basic_data['Company'] == 'hat', 'Kortech', basic_data['Company'])
basic_data['Company'] = np.where(basic_data['Company'] == 'j005', 'EDEX', basic_data['Company'])
basic_data['Company'] = np.where(basic_data['Company'] == 'hau', 'HAIM', basic_data['Company'])
joblist['Company'] = np.where(joblist['Company']== 'hau', 'HAIM', joblist['Company'])

basic_data['Company'] = np.where(basic_data['Personnel No.'] == 2168, 'hac', basic_data['Company'])

basic_data['Company'] = np.where(basic_data['Company']== 'hapm', 'Eden', basic_data['Company'])
joblist['Company'] = np.where(joblist['Company'] == 'hapm', 'Eden', joblist['Company'])

# Update 'Company' to 'PGESCO' when 'Previous SAP ID' matches specified values and 'Company' is 'HAH'
basic_data['Company'] = np.where(
    (basic_data['Company'].str.lower() == 'hah') & (basic_data['Previous SAP ID'].isin([10207370, 10207386, 10207506])),
    'PGESCO',
    basic_data['Company']  # Keep the original value if the condition is not met
)

basic_data['Company'] = np.where(
    (basic_data['Company'] == 'hac') & 
    (basic_data['Location Text'].str.contains('Steel', na=False)) & 
    (basic_data['Location Text'] != 'IIC Steel Making Plant'), 
    'HAS',  
    basic_data['Company']  
)

basic_data['Company'] = np.where(
    (basic_data['Company'] == 'hac') & (basic_data['Location Text'] == 'Wood Factory'),  
    'Wood',  
    basic_data['Company']  
)

# إنشاء عمود 'Concatenate - Position' بدون مسافات زائدة
basic_data['Concatenate - Position_2'] = basic_data['Company'] + basic_data['Position.1']


In [520]:
joblist.rename(columns={'Level':'Level_joblist'}, inplace=True)
joblist.rename(columns={'Employee Category':'Employee_Category_joblist'}, inplace=True)
joblist.drop_duplicates(inplace=True)
joblist.reset_index(inplace=True)
joblist

,level_0,index,Company,Position - Job List,Function,Department,Level_joblist,Level Category,Employee_Category_joblist,Concatenate - Position_2
0,0,0,3srm,chief executive officer,Top Management,Top Management,Top Management,White Collar,Core,3srmchief executive officer
1,1,1,3srm,operation manager,Operation,Operation,Managers,White Collar,Core,3srmoperation manager
2,2,2,3srm,area manager,Operation,Operation,Managers,White Collar,Core,3srmarea manager
3,3,3,3srm,plant head,Operation,Operation,Section Heads,White Collar,Core,3srmplant head
4,4,4,3srm,production engineer,Operation,Operation,Professionals,White Collar,Core,3srmproduction engineer
...,...,...,...,...,...,...,...,...,...,...
3714,3714,3714,kortech,supervisor - elec,Operation,Operation,Blue collar,Blue Collar,Core,kortechsupervisor - elec
3715,3715,3715,kortech,supervisor - mech,Operation,Operation,Blue collar,Blue Collar,Core,kortechsupervisor - mech
3716,3716,3716,kortech,projects control senior director,Project Controls,Project Controls,Directors,White Collar,Supportive,kortechprojects control senior director
3717,3717,3717,har,hr operation section head,HR Operation,Human Resources,Section Heads,White collar,Supportive,harhr operation section head


In [524]:
Withdrawn=basic_data[(basic_data['Status']=='withdrawn' ) & (basic_data ['Reason for Action']== 'transfer to sister company')]

Inactive=basic_data[(basic_data['Status']=='inactive' ) & (basic_data ['Action']== 'unpaid leave') ]


# Select the columns to merge from 'joblist'
columns_to_merge = ['Concatenate - Position_2', 'Function', 'Department', 'Level_joblist', 'Level Category', 'Employee_Category_joblist']

# Merge the selected columns from 'joblist' into 'Inactive' based on the ''Concatenate - Position' column
Inactive = Inactive.merge(joblist[columns_to_merge], on='Concatenate - Position_2', how='left')


Withdrawn_=basic_data[(basic_data['Status']=='withdrawn' ) & (basic_data ['Reason for Action']== 'transfer to sister company')].index
Inactive_=basic_data[(basic_data['Status']=='inactive' ) & (basic_data ['Action']== 'unpaid leave') ].index


basic_data.drop(Withdrawn_, axis=0, inplace=True)
basic_data.drop(Inactive_, axis=0, inplace=True)

basic_data.reset_index(drop=True, inplace=True)

In [526]:
Withdrawn=basic_data[(basic_data['Status']=='Withdrawn' ) & (basic_data ['Reason for Action']== 'Transfer to sister company')]

Inactive=basic_data[(basic_data['Status']=='Inactive' ) & (basic_data ['Action']== 'Unpaid Leave') ]

# Select the columns to merge from 'joblist'
columns_to_merge = ['Concatenate - Position_2', 'Function', 'Department', 'Level_joblist', 'Level Category', 'Employee_Category_joblist']

# Merge the selected columns from 'joblist' into 'Inactive' based on the ''Concatenate - Position' column
Inactive = Inactive.merge(joblist[columns_to_merge], on='Concatenate - Position_2', how='left')

Withdrawn_=basic_data[(basic_data['Status']=='Withdrawn' ) & (basic_data ['Reason for Action']== 'Transfer to sister company')].index
Inactive_=basic_data[(basic_data['Status']=='Inactive' ) & (basic_data ['Action']== 'Unpaid Leave') ].index


basic_data.drop(Withdrawn_, axis=0, inplace=True)
basic_data.drop(Inactive_, axis=0, inplace=True)

basic_data.reset_index(drop=True, inplace=True)

In [534]:
sap_position= basic_data[['Company','Position.1','Concatenate - Position_2']]
joblist.to_excel('joblist_p.xlsx')

In [483]:
sap_position= basic_data[['Company','Position.1','Concatenate - Position_2']]
# Select the columns to merge from 'joblist'
columns_to_merge = ['Concatenate - Position_2', 'Position.1']

# Merge the selected columns from 'joblist' into 'basic data' based on the ''Concatenate - Position' column
joblist = joblist.merge(sap_position[columns_to_merge], on='Concatenate - Position_2', how='left')
joblist['Concatenate - Position_2']= joblist['Company'] + joblist['Position.1']
joblist.drop_duplicates(inplace=True)
joblist.reset_index(inplace=True)
joblist.drop(['level_0','index'],inplace=True, axis=1)
joblist

,Company,Position - Job List,Function,Department,Level_joblist,Level Category,Employee_Category_joblist,concat_joblist,Position.1,Concatenate - Position_2
0,3srm,chief executive officer,Top Management,Top Management,Top Management,White Collar,Core,3srmchief executive officer,chief executive officer,3srmchief executive officer
1,3srm,operation manager,Operation,Operation,Managers,White Collar,Core,3srmoperation manager,operation manager,3srmoperation manager
2,3srm,area manager,Operation,Operation,Managers,White Collar,Core,3srmarea manager,area manager,3srmarea manager
3,3srm,plant head,Operation,Operation,Section Heads,White Collar,Core,3srmplant head,plant head,3srmplant head
4,3srm,production engineer,Operation,Operation,Professionals,White Collar,Core,3srmproduction engineer,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3714,kortech,supervisor - elec,Operation,Operation,Blue collar,Blue Collar,Core,kortechsupervisor - elec,NaN,NaN
3715,kortech,supervisor - mech,Operation,Operation,Blue collar,Blue Collar,Core,kortechsupervisor - mech,NaN,NaN
3716,kortech,projects control senior director,Project Controls,Project Controls,Directors,White Collar,Supportive,kortechprojects control senior director,NaN,NaN
3717,har,hr operation section head,HR Operation,Human Resources,Section Heads,White collar,Supportive,harhr operation section head,hr operation section head,harhr operation section head


In [498]:

# Select the columns to merge from 'joblist'
columns_to_merge = ['Concatenate - Position_2', 'Function', 'Department','Level_joblist', 'Level Category', 'Employee_Category_joblist']

# Merge the selected columns from 'joblist' into 'basic data' based on the ''Concatenate - Position' column
basic_data = basic_data.merge(joblist[columns_to_merge], on='Concatenate - Position_2', how='left')


basic_data.drop_duplicates(inplace=True)
basic_data.reset_index(inplace=True)

basic_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21821 entries, 0 to 21820
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   index                      21821 non-null  int64         
 1   Company                    21821 non-null  object        
 2   Personnel No.              21821 non-null  int64         
 3   Previous SAP ID            12879 non-null  float64       
 4   Employee Name              21821 non-null  object        
 5   Employee Arabic name       21816 non-null  object        
 6   Position                   21821 non-null  int64         
 7   Position.1                 21821 non-null  object        
 8   Concatenate - Position     21821 non-null  object        
 9   Service Type               0 non-null      float64       
 10  Job                        13759 non-null  object        
 11  Employee Category          20051 non-null  object        
 12  Mana

In [469]:
missng= basic_data[pd.isna(basic_data['Function'])]
missng[['Company','Position.1']]

,Company,Position.1
105,Wood,driver
155,hac,integration: default position
210,HAS,security officer
229,HAS,human resource manager
296,hac,integration: default position
...,...,...
22943,Kortech,site engineer section head - mech
22944,Kortech,estimation engineer
22945,Kortech,senior tendering manager
22946,Kortech,sap support specilaist


In [ ]:
# Consolidating all transformations
basic_data['Company'] = np.where(
    # Update Company to 'Eden-Blue Collars' or 'Eden-White Collars'
    (basic_data['Company'] == 'eden') & (basic_data['Level'].isin(['Ancillary', 'Blue collar'])),
    'Eden-Blue Collars',
    np.where(
        (basic_data['Company'] == 'eden') & (basic_data['Level'].isin(['Professionals', 'Sr. Managers', 'Section Heads', 'Managers', 'Directors', 'Top Management'])),
        'Eden-White Collars',
        np.where(
            # Update Company to 'EDEX' for specific Organizational Units
            (basic_data['Company'] == 'eden') & (basic_data['Organizational Unit'].isin(['Dabaa Expedition - EDEN NON-Manual', 'Dabaa Expedition - EDEN - Manual'])),
            'EDEX',
            np.where(
                # Update Company to 'HAC-Int.' for specific Sector Text
                (basic_data['Company'] == 'hac') & (basic_data['Sector Text'] == 'International Operations'),
                'HAC-Int.',
                basic_data['Company']  # Keep original value
            )
        )
    )
)


# Create Service Type column based on Function
basic_data['Service Type'] = np.where(
    (basic_data['Function'] == 'Hard Service'),
    'Hard Service',
    np.where(
        (basic_data['Function'] == 'Soft Service'),
        'Soft Service',
        np.nan  # Default to NaN
    )
)



level_order = {
    "Top Management": 1,
    "Directors": 2,
    "Sr. Managers": 3,
    "Managers": 4,
    "Section Heads": 5,
    "Professionals": 6,
    "Ancillary": 7,
    "Blue Collar": 8
}
basic_data['Level Sorting'] = basic_data['Level_x'].map(level_order)


# Update Organizational Unit for 'sua' and Special Needs
basic_data['Organizational Unit'] = np.where(
    (basic_data['Company'] == 'sua') & (basic_data['Organizational Unit'] == 'Special Needs'),
    basic_data['Function'],
    basic_data['Organizational Unit']
)


In [10]:


# Update Employee Category_x for 'sua' and Special Needs
basic_data['Employee Category_x'] = np.where(
    (basic_data['Company'] == 'sua') & (basic_data['Organizational Unit'] == 'Special Needs'),
    'Nationals',
    basic_data['Employee Category_x']
)

# Update Status for 'sua' and Nationals
basic_data['Status'] = np.where(
    (basic_data['Company'] == 'sua') & (basic_data['Employee Category_x'] == 'Nationals'),
    'Nationals',
    basic_data['Status']
)

# Update Organizational Unit for 'sua' and Special Needs
basic_data['Organizational Unit'] = np.where(
    (basic_data['Company'] == 'sua') & (basic_data['Organizational Unit'] == 'Special Needs'),
    basic_data['Function'],
    basic_data['Organizational Unit']
)



# Debugging output: Check for specific conditions
check = basic_data[['Personnel No.', 'Company', 'Organizational Unit', 'Position.1', 'Employee Category_x']]
filtered_check = check[(check['Company'] == 'sua') & (check['Employee Category_x'] == 'Nationals')]

# Display filtered results
filtered_check

,Personnel No.,Company,Organizational Unit,Position.1,Employee Category_x
10577,12000252,sua,Purchasing,purchasing representative,Nationals
10578,12000253,sua,Purchasing,purchasing representative,Nationals
10579,12000254,sua,Administration,administration assistant,Nationals
10580,12000255,sua,Administration,administration assistant,Nationals
10581,12000256,sua,Purchasing,purchasing representative,Nationals
10583,12000258,sua,Administration,administration assistant,Nationals
10584,12000259,sua,Purchasing,purchasing representative,Nationals
10585,12000260,sua,Administration,administration assistant,Nationals
10586,12000261,sua,Administration,administration assistant,Nationals
10587,12000262,sua,Purchasing,purchasing representative,Nationals


In [11]:
basic_data['Employee Category_x'] = np.where(
    (basic_data['Company'] == 'sua') & (basic_data['Organizational Unit'] == 'Special Needs'), 
    'Nationals', 
    basic_data['Employee Category_x'] 
)

basic_data['Status'] = np.where(
    (basic_data['Company'] == 'sua') & (basic_data['Employee Category_x'] == 'Nationals'), 
    'Nationals', 
    basic_data['Status'] 
)


basic_data['Organizational Unit'] = np.where(
    (basic_data['Company'] == 'sua') & (basic_data['Organizational Unit'] == 'Special Needs'), 
    basic_data['Function'],  
    basic_data['Organizational Unit']  
)

check=basic_data[['Personnel No.','Company', 'Organizational Unit', 'Position.1', 'Employee Category_x']]
check[(check['Company']=='sua') & (check['Employee Category_x']=='Nationals') ]

,Personnel No.,Company,Organizational Unit,Position.1,Employee Category_x
10577,12000252,sua,Purchasing,purchasing representative,Nationals
10578,12000253,sua,Purchasing,purchasing representative,Nationals
10579,12000254,sua,Administration,administration assistant,Nationals
10580,12000255,sua,Administration,administration assistant,Nationals
10581,12000256,sua,Purchasing,purchasing representative,Nationals
10583,12000258,sua,Administration,administration assistant,Nationals
10584,12000259,sua,Purchasing,purchasing representative,Nationals
10585,12000260,sua,Administration,administration assistant,Nationals
10586,12000261,sua,Administration,administration assistant,Nationals
10587,12000262,sua,Purchasing,purchasing representative,Nationals


In [ ]:
sector_list = [
    "Support Division", "Civil Sector 4", "Infrastructure Sector", "Power Sector",
    "MEP Sector", "Civil Sector 2", "Civil Sector 7", "Civil Sector 1", "Civil Sector 10",
    "JV Sector", "Civil Sector 6", "Sector 0", "Civil Sector 9", "Equipment",
    "Projects Sector", "Eden", "Bridges Sector", "Operation Sector 2", "Operation Sector 1",
    "Operation", "Quality Control", "Bridges", "Roads Sector", "Roads", "Central Warehouse",
    "Railway Track Sector", "Commercial  &Warehouse", "Operation and Maintenance", "HSE",
    "HILTI", "Projects Management", "Micro Tunneling", "MT", "239- AVN1200", "HDD",
    "AVN - 1600", "AVN - 600", "AVN - 1200", "AVN - 800", "Horizontal Directional Drilling",
    "Civil Sector", "Human Resources", "Contracts", "Steel Factory", "Steel Sector",
    "EDEX", "Supply Chain"
]




# Consolidating all transformations
basic_data['personal_area_correct'] = np.where(
    (basic_data['Location Text'].isin(['Block Factory','Wood Factory','Steel Factory','Flex Park Warehouse - Steel','DNPP-Steel',
 'M.FBL Expan-P-P-1S-W - Steel','SamsungTabletFactory - Steel'])),
    'Factories',
    np.where(
         (basic_data['Sector Text'].isin(['Libya Projects','International Operations','Infrastructure Sector'])),
        'Int. Operation',
        np.where(
            # Update Company to 'EDEX' for specific Organizational Units
            (basic_data['Sector Text'].isin(sector_list)),
            'Operation',
            
    )
)

In [ ]:
basic_data['Libya Projects','International Operations','Infrastructure Sector']

In [ ]:
           # Create the list of locations
sector_list = [
    "Support Division", "Civil Sector 4", "Infrastructure Sector", "Power Sector",
    "MEP Sector", "Civil Sector 2", "Civil Sector 7", "Civil Sector 1", "Civil Sector 10",
    "JV Sector", "Civil Sector 6", "Sector 0", "Civil Sector 9", "Equipment",
    "Projects Sector", "Eden", "Bridges Sector", "Operation Sector 2", "Operation Sector 1",
    "Operation", "Quality Control", "Bridges", "Roads Sector", "Roads", "Central Warehouse",
    "Railway Track Sector", "Commercial  &Warehouse", "Operation and Maintenance", "HSE",
    "HILTI", "Projects Management", "Micro Tunneling", "MT", "239- AVN1200", "HDD",
    "AVN - 1600", "AVN - 600", "AVN - 1200", "AVN - 800", "Horizontal Directional Drilling",
    "Civil Sector", "Human Resources", "Contracts", "Steel Factory", "Steel Sector",
    "EDEX", "Supply Chain"
]


# Use the list in the code
filtered_data = basic_data[basic_data['Sector Text'].isin(locations_list)]


In [21]:
from datetime import datetime

#Convert 'Hiring Date' column to datetime format
basic_data['Hiring Date'] = pd.to_datetime(basic_data['Hiring Date'], errors='coerce')

# Function to calculate years and months between the hiring date and today
def calculate_years_months(date):
    today = datetime.today()
    # Calculate years difference
    years = today.year - date.year - ((today.month, today.day) < (date.month, date.day))
    # Calculate months difference after accounting for full years
    months = today.month - date.month - (today.day < date.day)
    if months < 0:
        months += 12
    return f"{years} years, {months} Months"

# Apply the function to create a new 'Years of Service' column
basic_data['Years of Service'] = basic_data['Hiring Date'].apply(calculate_years_months)


# Define a function to categorize Tenurity based on Years of Service
def categorize_tenurity(years_of_service):
    years = int(years_of_service.split()[0])  # Extract the year as an integer
    
    if years < 1:
        return "Probation" if "Months" in years_of_service and int(years_of_service.split()[2]) <= 6 else "7 - 12 Months"
    elif years < 2:
        return "1 - 2 Years"
    elif years < 5:
        return "2 - 5 Years"
    elif years < 10:
        return "5 - 10 Years"
    else:
        return "10+ Years"

# Apply the categorization function
basic_data['Tenurity'] = basic_data['Years of Service'].apply(categorize_tenurity)

# Map Tenurity to sorting values for the "Tenurity Sorting" column
tenurity_order = {
    "Probation": 1,
    "4 - 6 Months": 2,
    "7 - 12 Months": 3,
    "1 - 2 Years": 4,
    "2 - 5 Years": 5,
    "5 - 10 Years":6,
    "10+ Years": 7
}
basic_data['Tenurity Sorting'] = basic_data['Tenurity'].map(tenurity_order)

#basic_data.drop(['Job','ACTION MEMO','Managerial Duties',],inplace=True, axis=1)



In [23]:
basic_data['C-Function'] = basic_data['Company'].astype(str) + basic_data['Function'].astype(str)
basic_data['C-Department'] = basic_data['Company'].astype(str) + basic_data['Department'].astype(str)
basic_data['C-Level'] = basic_data['Company'].astype(str) + basic_data['Level'].astype(str)

level_order = {
    "Top Management": 1,
    "Directors": 2,
    "Sr. Managers": 3,
    "Managers": 4,
    "Section Heads": 5,
    "Professionals": 6,
    "Ancillary": 7,
    "Blue Collar": 8
}
basic_data['Level Sorting'] = basic_data['Level'].map(level_order)

In [25]:
# Define a function to categorize values in the new column "Eng. & Non. Eng."
def categorize_engineer(row):
    
    # If "Engineering" is found in "Major Education", return "Engineer"
    if pd.notna(row['Major Education']) and "Engineering" in row['Major Education']:
        return "Engineer"

    # If "Engineer" is found in "Position", return "Engineer"
    elif pd.notna(row['Position.1']) and "engineer" in row['Position.1']:
        return "Engineer"
    # If "Major Education" is missing (NaN), return "No Education"
    elif pd.isna(row['Major Education']):
        return "No Education"
    # Otherwise, return "Non Engineer"
    else:
        return "Non Engineer"

# Apply the function to create the new column "Eng. & Non. Eng."
basic_data['Eng. & Non. Eng.'] = basic_data.apply(categorize_engineer, axis=1)

basic_data['C-Eduation'] = basic_data['Company'].astype(str) + basic_data['Eng. & Non. Eng.'].astype(str)

In [37]:
basic_data['Eng. & Non. Eng.']

0        Non Engineer
1            Engineer
2            Engineer
3        Non Engineer
4        Non Engineer
             ...     
17020        Engineer
17021        Engineer
17022        Engineer
17023    Non Engineer
17024        Engineer
Name: Eng. & Non. Eng., Length: 17023, dtype: object

In [27]:
# Display the result to check
a= basic_data[basic_data['Major Education']=='Engineering'] [['Position.1','Major Education' ,'Eng. & Non. Eng.']]
#a[(a['Position.1'].str.contains('Engineer')==True) & (a['Major Education']!= 'Engineering')]
a

,Position.1,Major Education,Eng. & Non. Eng.
1,senior estimation manager,Engineering,Engineer
2,workshop manager,Engineering,Engineer
5,quantity surveyor manager,Engineering,Engineer
7,chief commercial officer,Engineering,Engineer
9,chief operation officer,Engineering,Engineer
...,...,...,...
17019,senior technical office engineer - elec,Engineering,Engineer
17020,site engineer section head - mech,Engineering,Engineer
17021,estimation engineer,Engineering,Engineer
17022,senior tendering manager,Engineering,Engineer


In [29]:
# 1. Create 'Age Risk' column based on 'Age of employee'
basic_data['Age Risk'] = basic_data['Age of employee'].apply(lambda age: "Age Risk" if age >= 58 else "No Risk")

# 2. Extract 'Year of Birth' from 'Date of Birth' column
# Ensure 'Date of Birth' is in datetime format
basic_data['Date of Birth'] = pd.to_datetime(basic_data['Date of Birth'], errors='coerce')
basic_data['Year of Birth'] = basic_data['Date of Birth'].dt.year

# 3. Define Generation mapping based on 'Year of Birth'
generation_mapping = {
    range(1928, 1946): "Traditionalists",
    range(1946, 1965): "Baby Boomers",
    range(1965, 1981): "X",
    range(1981, 1997): "Y",
    range(1997, 2016): "Z"
}

# Function to determine generation based on 'Year of Birth'
def get_generation(year):
    for year_range, generation in generation_mapping.items():
        if year in year_range:
            return generation
    return np.nan  # Return NaN if the year doesn't match any generation range

# Apply the function to create the 'Generation' column
basic_data['Generation'] = basic_data['Year of Birth'].apply(get_generation)

# 4. Define 'Generation Sorting' based on the generation values
generation_sorting = {
    "Traditionalists": 1,
    "Baby Boomers": 2,
    "X": 3,
    "Y": 4,
    "Z": 5
}

# Map 'Generation' column to 'Generation Sorting'
basic_data['Generation Sorting'] = basic_data['Generation'].map(generation_sorting)

basic_data['C-Generation'] = basic_data['Company'].astype(str) + basic_data['Generation'].astype(str)

# Display the updated DataFrame to verify results
basic_data[['Age of employee', 'Age Risk', 'Date of Birth', 'Year of Birth', 'Generation', 'Generation Sorting']]

,Age of employee,Age Risk,Date of Birth,Year of Birth,Generation,Generation Sorting
0,65,Age Risk,1959-01-01,1959,Baby Boomers,2.0
1,66,Age Risk,1958-07-23,1958,Baby Boomers,2.0
2,73,Age Risk,1951-11-17,1951,Baby Boomers,2.0
3,63,Age Risk,1961-09-19,1961,Baby Boomers,2.0
4,66,Age Risk,1958-06-13,1958,Baby Boomers,2.0
...,...,...,...,...,...,...
17020,33,No Risk,1991-06-17,1991,Y,4.0
17021,26,No Risk,1998-06-23,1998,Z,5.0
17022,52,No Risk,1972-03-08,1972,X,3.0
17023,22,No Risk,2002-08-29,2002,Z,5.0


In [31]:
basic_data.drop_duplicates(inplace = True)

In [37]:
def Data_Overview (basic_data):
    print(f"- Dataset Shape: {basic_data.shape}")
    
    print("--"*20)
    
    print(f'- Daplication data: {basic_data.duplicated().sum()}')
    
    print("--"*20)
    
    summary = pd.DataFrame(basic_data.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = basic_data.isnull().sum().values 
    summary['Missing %'] = round(basic_data.isnull().mean()*100,2).values    
    summary['Uniques'] = basic_data.nunique().values
    return summary.style.background_gradient(cmap ='RdBu_r')

Data_Overview(basic_data)

- Dataset Shape: (17000, 51)
----------------------------------------
- Daplication data: 0
----------------------------------------


,Name,dtypes,Missing,Missing %,Uniques
0,Company,object,0,0.000000,27
1,Personnel No.,int64,0,0.000000,16523
2,Previous SAP ID,float64,9178,53.990000,7685
3,Employee Name,object,0,0.000000,16432
4,Employee Arabic name,object,5,0.030000,16383
5,Position,int64,0,0.000000,16522
6,Position.1,object,0,0.000000,1456
7,Job,object,8243,48.490000,860
8,Employee Category_x,object,1748,10.280000,3
9,Managerial Duties,object,16865,99.210000,1


In [33]:
basic_data[pd.isna(basic_data['Level Sorting'])]

,Company,Personnel No.,Previous SAP ID,Employee Name,Employee Arabic name,Position,Position.1,Job,Employee Category_x,Managerial Duties,...,C-Department,C-Level,Level Sorting,Eng. & Non. Eng.,C-Eduation,Age Risk,Year of Birth,Generation,Generation Sorting,C-Generation
6,hac,58,10000141.0,MOSAAD ABD ELAZIZ MOHAMED SOLIMAN,مسعد عبد العزيز محمد سليمان,1030,technician,Technician,Supportive,NaN,...,hacSupport Division,hacBlue collar,NaN,Non Engineer,hacNon Engineer,Age Risk,1960,Baby Boomers,2.0,hacBaby Boomers
10,hac,68,10000250.0,TAREK TAWFIK MOHAMED EBIED,طارق توفيق محمد عبيد,64199,foreman,Foreman,Core,NaN,...,hacOperation,hacBlue collar,NaN,Non Engineer,hacNon Engineer,No Risk,1966,X,3.0,hacX
12,hac,70,10000256.0,AHMED MOAWAD MOUSA ABD ELAZIZ,احمد معوض موسى عبد العزيز,1042,senior mechanical technician,Senior Technician,Supportive,NaN,...,hacSupport Division,hacBlue collar,NaN,Non Engineer,hacNon Engineer,No Risk,1970,X,3.0,hacX
23,hac,88,10000606.0,TAHER MAKRAM ZAKI MOHAMED,طاهر مكرم زكى محمد,64708,mechanical technician,Technician,Supportive,NaN,...,hacSupport Division,hacBlue collar,NaN,Non Engineer,hacNon Engineer,No Risk,1977,X,3.0,hacX
34,hac,104,10000701.0,MOHAMED ABD ELSATAR ELSAYED ALI,محمد عبد الستار السيد علي,50012434,technician,Technician,Supportive,NaN,...,hacSupport Division,hacBlue collar,NaN,Non Engineer,hacNon Engineer,No Risk,1970,X,3.0,hacX
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16811,kortech,66001311,NaN,MOHAMED GAMAL HOSNY IBRAHIM,محمد جمال حسنى ابراهيم,48505,electrical foreman,MEP Foreman,Core,NaN,...,kortechOperation,kortechBlue collar,NaN,Non Engineer,kortechNon Engineer,No Risk,1993,Y,4.0,kortechY
16816,kortech,66001319,NaN,WAHED ELSAYED FAWZY MEGAHED,وحيد السيد فوزى مجاهد,57301,electrical foreman,NaN,Core,NaN,...,kortechOperation,kortechBlue collar,NaN,Non Engineer,kortechNon Engineer,No Risk,1990,Y,4.0,kortechY
16957,kortech,66001541,NaN,MAHMOUD SABER REZK MOHAMED ELWAKEEL,محمود صابر رزق محمد الوكيل,74133,maintenance foreman,Foreman,Supportive,NaN,...,kortechOperation,kortechBlue collar,NaN,Non Engineer,kortechNon Engineer,No Risk,1991,Y,4.0,kortechY
16967,kortech,66001551,NaN,HUSSIEN ELSAYED HUSSIEN SAND,حسين السيد حسين سند,74134,maintenance foreman,Foreman,Supportive,NaN,...,kortechOperation,kortechBlue collar,NaN,Non Engineer,kortechNon Engineer,No Risk,1996,Y,4.0,kortechY


In [39]:
basic_data.columns

Index(['Company', 'Personnel No.', 'Previous SAP ID', 'Employee Name',
       'Employee Arabic name', 'Position', 'Position.1', 'Job',
       'Employee Category_x', 'Managerial Duties', 'Manager Pr.No.',
       'Manager Name', 'Status', 'Hiring Date', 'Action Date', 'Action',
       'Reason for Action', 'Org. Unit', 'Organizational Unit',
       'Location Text', 'Sector Text', 'Department Text',
       'Personnel Area - Correct', 'Personnel Subarea', 'Employee Group',
       'Level', 'From', 'Major Education', 'Branch of study', 'Gender text',
       'Date of Birth', 'Age of employee', 'Acting as',
       'Concatenate - Position_2', 'Function', 'Department', 'Level Category',
       'Employee Category_y', 'Service Type', 'Years of Service', 'Tenurity',
       'Tenurity Sorting', 'C-Function', 'C-Department', 'C-Level',
       'Level Sorting', 'Eng. & Non. Eng.', 'C-Eduation', 'Age Risk',
       'Year of Birth', 'Generation', 'Generation Sorting', 'C-Generation'],
      dtype='object')

In [35]:
basic_data = basic_data.apply(lambda x: x.str.title() if x.dtype == "object" else x)
basic_data

,Company,Personnel No.,Previous SAP ID,Employee Name,Employee Arabic name,Position,Position.1,Job,Employee Category_x,Managerial Duties,...,C-Department,C-Level,Level Sorting,Eng. & Non. Eng.,C-Eduation,Age Risk,Year of Birth,Generation,Generation Sorting,C-Generation
0,Hac,50,10000022.0,Effat Soliman Mohamed Sayed,عفت سليمان محمد سيد,1022,Chief Accountant,Chief Accountant,Supportive,NaN,...,Hacfinance,Hacsection Heads,5.0,Non Engineer,Hacnon Engineer,Age Risk,1959,Baby Boomers,2.0,Hacbaby Boomers
1,Hac,52,10000032.0,Abd Elaziz Mohamed Ibrahim Ghazawy,عبد العزيز محمد ابراهيم غزاوي,1024,Senior Estimation Manager,Senior Estimation Manager,Supportive,NaN,...,Hactendering,Hacmanagers,4.0,Engineer,Hacengineer,Age Risk,1958,Baby Boomers,2.0,Hacbaby Boomers
2,Hac,54,10000097.0,Ali Omar Abdallah Mohamed,على عمر عبد الله محمد,1026,Workshop Manager,Workshop Manager,Supportive,NaN,...,Hacsupport Division,Hacmanagers,4.0,Engineer,Hacengineer,Age Risk,1951,Baby Boomers,2.0,Hacbaby Boomers
3,Hac,55,10000114.0,Ashraf Moharam Ahmed Othman,اشرف محرم احمد عثمان,66141,Administration Coordinator,Administration Coordinator,Supportive,NaN,...,Hacadministration,Hacprofessionals,6.0,Non Engineer,Hacnon Engineer,Age Risk,1961,Baby Boomers,2.0,Hacbaby Boomers
4,Hac,56,10000118.0,Wahed Radwan Ahmed Eleter,وحيد رضوان احمد العتر,1028,Accounting Manager,Accounting Manager,Supportive,NaN,...,Hacfinance,Hacmanagers,4.0,Non Engineer,Hacnon Engineer,Age Risk,1958,Baby Boomers,2.0,Hacbaby Boomers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17020,Kortech,66001607,NaN,Omar Hussein Mahmoud Gad,عمر حسين محمود جاد,3470,Site Engineer Section Head - Mech,Mep Section Head,Core,NaN,...,Kortechoperation,Kortechsection Heads,5.0,Engineer,Kortechengineer,No Risk,1991,Y,4.0,Kortechy
17021,Kortech,66001608,NaN,Islam Mohamed Sayed Ibrahim Ayad,اسلام محمد سيد ابراهيم عياد,50016327,Estimation Engineer,Estimation Engineer,Core,NaN,...,Kortechtendering,Kortechprofessionals,6.0,Engineer,Kortechengineer,No Risk,1998,Z,5.0,Kortechz
17022,Kortech,66001609,NaN,Mohamed Abdallah Mohamed Saad Seoud,محمد عبدالله محمد سعد سعود,50016806,Senior Tendering Manager,Senior Tendering Manager,Supportive,NaN,...,Kortechtendering,Kortechsr. Managers,3.0,Engineer,Kortechengineer,No Risk,1972,X,3.0,Kortechx
17023,Kortech,66001610,NaN,Mahmoud Sameh Mahmoud Elsayed Metwaly Am,محمود سامح محمود السيد متولى عامر,3928,Sap Support Specilaist,Sap Support Specilaist,Supportive,NaN,...,Kortechdigital Transformation,Kortechprofessionals,6.0,Non Engineer,Kortechnon Engineer,No Risk,2002,Z,5.0,Kortechz


In [41]:
# Specify the file name
file_name = '17_NOV_Cleaning .xlsx'

# Save to Excel with multiple sheets
with pd.ExcelWriter(file_name, engine='xlsxwriter') as writer:
    basic_data.to_excel(writer, sheet_name='basic_data', index=False)
    Withdrawn.to_excel(writer, sheet_name='Withdrawn', index=False)
    Inactive.to_excel(writer, sheet_name='Inactive', index=False)
    
    
    
    

print(f'File saved as {file_name}')

File saved as 17_NOV_Cleaning .xlsx


## Action list 

In [127]:
action_list = pd.read_excel(r'C:\Users\salma.elsayed\Desktop\basic data\3Nov_analysis.xlsx', sheet_name='action list')
action_list.head(2)

,company code,Action Date,Action,Employee ID,Name,New Location,New Sector,Pers.area,New Subarea,EE group,EESubgroup,New organization,New position,New job,Old Subarea,Old location,Old Sector,Old organization,Old position,Old job
0,HAC,05.10.2008,Hiring,9270,Radwan Abdelhalim Abdulla,Abu Sultan Touristic Centre Project,NaN,Operation,South upper Egy,Operation,Ancillary,Human Resources,HR Coordinator,NaN,Head Office,NaN,NaN,NaN,NaN,NaN
1,HAC,01.01.2012,Hiring,9264,Reda Mohamed Gad,Abu Sultan Touristic Centre Project,NaN,Operation,South upper Egy,Operation,Professionals,Security,Security Supervisor,NaN,Head Office,NaN,NaN,NaN,NaN,NaN


In [128]:
def Data_Overview (action_list):
    print(f"- Dataset Shape: {action_list.shape}")
    
    print("--"*20)
    
    print(f'- Daplication data: {action_list.duplicated().sum()}')
    
    print("--"*20)
    
    summary = pd.DataFrame(action_list.dtypes,columns=['dtypes'])
    summary = summary.reset_index()
    summary['Name'] = summary['index']
    summary = summary[['Name','dtypes']]
    summary['Missing'] = action_list.isnull().sum().values 
    summary['Missing %'] = round(action_list.isnull().mean()*100,2).values    
    summary['Uniques'] = action_list.nunique().values
    return summary.style.background_gradient(cmap ='RdBu_r')

Data_Overview(action_list)

- Dataset Shape: (90119, 20)
----------------------------------------
- Daplication data: 29
----------------------------------------


,Name,dtypes,Missing,Missing %,Uniques
0,company code,object,0,0.000000,22
1,Action Date,object,0,0.000000,1723
2,Action,object,0,0.000000,23
3,Employee ID,int64,0,0.000000,24331
4,Name,object,0,0.000000,24057
5,New Location,object,1643,1.820000,1054
6,New Sector,object,10448,11.590000,169
7,Pers.area,object,0,0.000000,20
8,New Subarea,object,0,0.000000,48
9,EE group,object,0,0.000000,4


In [129]:
action_list.rename(columns={'Employee ID': 'Personnel No.'}, inplace=True)

action_list.drop_duplicates(inplace = True)

In [119]:
action_list['company code'] = np.where(action_list['company code'] == 'HAT', 'Kortech', action_list['company code'])

action_list['company code'] = np.where(action_list['company code'] == 'J005', 'EDEX', action_list['company code'])

action_list['company code'] = np.where(action_list['company code'] == 'HAU', 'HAIM', action_list['company code'])

action_list['company code'] = np.where(action_list['Personnel No.'] == 2168, 'hac', action_list['company code'])

action_list['company code'] = np.where(action_list['company code'] == 'HAPM', 'Eden', action_list['company code'])

# Update 'Company' to 'PGESCO' when 'Previous SAP ID' matches specified values and 'Company' is 'HAH'
action_list['company code'] = np.where(
    (action_list['company code'] == 'HAH') & (action_list['Personnel No.'].isin([5000222, 5000223, 5000228])),
    'PGESCO',
    action_list['company code']  # Keep the original value if the condition is not met
)


action_list['company code'] = np.where(
    (action_list['company code'] == 'HAC') & 
    (action_list['New Location'].str.contains('Steel', na=False)) & 
    (action_list['New Location'] != 'IIC Steel Making Plant'), 
    'HAS',  
    action_list['company code']  
)
action_list['company code'] = np.where(
    (action_list['company code'] == 'HAC') & (action_list['New Location'] == 'Wood Factory'),  
    'Wood',  
    action_list['company code']  
)

# Convert 'Action Date' column to datetime format
action_list['Action Date'] = pd.to_datetime(action_list['Action Date'], errors='coerce')


action_list.drop(['Old Subarea', 'Old location', 'Old Sector', 'Old organization',
       'Old position', 'Old job','New job'],axis=1,inplace=True)

# List of actions to exclude
actions_to_exclude = [
    'Stop From Work', 'Back to Work', 'Demotion', 'Correction', 
    'Reactivate employment contract', 'Stop the employment contract', 
    'End Mission', 'Start Mission', 'Retention'
]

# Filter out rows where 'Action' is in the actions_to_exclude list
action_list = action_list[~action_list['Action'].isin(actions_to_exclude)]


# List of values to replace and their replacement
values_to_replace = [
    'Transfer between departments', 
    'Transfer between projects', 
    'Transfer(Projects&Departments)', 
    'Transfer between (JV&N)'
]

# Replace specified values with 'Internal Transfers'
action_list['Action'] = action_list['Action'].replace(values_to_replace, 'Internal Transfers')


action_list['Action'] = action_list['Action'].replace('Tran (Egypt & Int)', 'External Transfers')

# Create a new column 'In/Out' with multiple conditions
action_list['In/Out'] = np.where(
    action_list['Action'].isin(['Hiring', 'Rehire', 'Transfer Between Companies']), 'Fulfillment',
    np.where(action_list['Action'].isin(['Termination', 'Retirment', 'Resignation']), 'Separation', np.nan)
)


action_list['company code']=action_list['company code'].str.strip().str.lower() 
action_list['New position']=action_list['New position'].str.strip().str.lower()
action_list['Concatenate - Position_2']= action_list['company code'].str.strip() + action_list['New position'].str.strip()

In [121]:
# Select the columns to merge from 'joblist'
columns_to_merge = ['Concatenate - Position_2', 'Function', 'Department', 'Level', 'Level Category', 'Employee Category']

# Merge the selected columns from 'joblist' into 'basic data' based on the ''Concatenate - Position' column
action_list = action_list.merge(joblist[columns_to_merge], on='Concatenate - Position_2', how='left')

action_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 411583 entries, 0 to 411582
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   company code              411583 non-null  object        
 1   Action Date               242172 non-null  datetime64[ns]
 2   Action                    411583 non-null  object        
 3   Personnel No.             411583 non-null  int64         
 4   Name                      411583 non-null  object        
 5   New Location              407405 non-null  object        
 6   New Sector                235243 non-null  object        
 7   Pers.area                 411583 non-null  object        
 8   New Subarea               411583 non-null  object        
 9   EE group                  411583 non-null  object        
 10  EESubgroup                411583 non-null  object        
 11  New organization          407106 non-null  object        
 12  Ne

In [123]:
missng= action_list[pd.isna(action_list['Function'])]
missng['Concatenate - Position_2'].nunique()


1621

In [125]:
# Apply conditions to update the 'Company' column based on 'EE Subgroup' and when 'Company' is 'Eden'
action_list['company code'] = np.where(
    (action_list['company code'] == 'eden') & (action_list['EESubgroup'].isin(['Ancillary', 'Blue collar'])),
    'Eden-Blue Collars',
    np.where(
        (action_list['company code'] == 'eden') & (action_list['EESubgroup'].isin(['Professionals', 'Sr. Managers', 'Section Heads', 'Managers', 'Directors', 'Top Management'])),
        'Eden-White Collars',
        action_list['company code']  # Keep the original value if conditions are not met
    )
)

# Apply conditions to update the 'Company' column based on 'EE Subgroup' and when 'Company' is 'Eden'
basic_data['Company'] = np.where(
    (basic_data['Company'] == 'eden') & (basic_data['Organizational Unit'].isin(['Dabaa Expedition - EDEN NON-Manual', 'Dabaa Expedition - EDEN - Manual'])),
    'EDEX',
    basic_data['Company']  # Keep the original value if conditions are not met
    )
)

# Update 'Company' to 'HAC-Int.' when 'Sector Text' is 'International Operations' and 'Company' is 'HAC'
action_list['company code'] = np.where(
    (action_list['company code'] == 'hac') & (action_list['New Sector'] == 'International Operations'),
    'HAC-Int.',
    action_list['company code']  # Keep the original value if the condition is not met
)




action_list['New organization'] = np.where(
    (action_list['company code'] == 'sua') & (action_list['New organization'] == 'Special Needs'), 
    action_list['Function'],  
    action_list['New organization']  
)

In [109]:
# List of targeted action types
ti = ['Rehire', 'Promotion', 'Resignation', 'Retirement', 'Termination', 'Unpaid Leave']

# Function to calculate years and months between two dates
def calculate_years_months_between(action_date, hiring_date, action_type):
    # Check if the action type is in the list
    if action_type in ti:
        # Ensure both dates are not missing (NaN)
        if pd.notna(action_date) and pd.notna(hiring_date):
            # Calculate the difference in years
            years = action_date.year - hiring_date.year - ((action_date.month, action_date.day) < (hiring_date.month, hiring_date.day))
            # Calculate the difference in months, adjusting if necessary
            months = action_date.month - hiring_date.month - (action_date.day < hiring_date.day)
            if months < 0:
                months += 12
            return f"{years} years, {months} Months"
        else:
            return np.nan  # Return NaN if either date is missing
    else:
        return np.nan  # Return NaN if the action type is not in the list

# Apply the function to the relevant columns
action_list['Years of Service'] = action_list.apply(
    lambda row: calculate_years_months_between(row['Action Date'], row['Hiring Date'], row['Action']),
    axis=1
)

# Define a function to categorize Tenurity based on Years of Service
def categorize_tenurity(years_of_service):
    if pd.isnull(years_of_service):  # Handle NaN values
        return np.nan
    years = int(years_of_service.split()[0])  # Extract the year as an integer
    
    if years < 1:
        return "Probation" if "Months" in years_of_service and int(years_of_service.split()[2]) <= 6 else "7 - 12 Months"
    elif years < 2:
        return "1 - 2 Years"
    elif years < 5:
        return "2 - 5 Years"
    elif years < 10:
        return "5 - 10 Years"
    else:
        return "10+ Years"

# Apply the categorization function
action_list['Tenurity'] = action_list['Years of Service'].apply(categorize_tenurity)

# Map Tenurity to sorting values for the "Tenurity Sorting" column
tenurity_order = {
    "Probation": 1,
    "4 - 6 Months": 2,
    "7 - 12 Months": 3,
    "1 - 2 Years": 4,
    "2 - 5 Years": 5,
    "5 - 10 Years": 6,
    "10+ Years": 7
}
action_list['Tenurity Sorting'] = action_list['Tenurity'].map(tenurity_order)


# Define a function to categorize values in the new column "Eng. & Non. Eng."
def categorize_engineer(row):
   # If "Engineering" is found in "Major Education", return "Engineer"
    if pd.notna(row['Major Education']) and "Engineering" in row['Major Education']:
        return "Engineer"

    # If "Engineer" is found in "Position", return "Engineer"
    elif pd.notna(row['Position.1']) and "engineer" in row['Position.1']:
        return "Engineer"
    # If "Major Education" is missing (NaN), return "No Education"
    elif pd.isna(row['Major Education']):
        return "No Education"
    # Otherwise, return "Non Engineer"
    else:
        return "Non Engineer"

# Apply the function to create the new column "Eng. & Non. Eng."
action_list['Eng. & Non. Eng.'] = action_list.apply(categorize_engineer, axis=1)


action_list['C-Eduation'] = action_list['Company'].astype(str) + merged_data['Eng. & Non. Eng.'].astype(str)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415782 entries, 0 to 415781
Data columns (total 35 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   company code              415782 non-null  object        
 1   Action Date               244029 non-null  datetime64[ns]
 2   Action                    415782 non-null  object        
 3   Personnel No.             415782 non-null  int64         
 4   Name                      415782 non-null  object        
 5   New Location              411597 non-null  object        
 6   New Sector                238719 non-null  object        
 7   Pers.area                 415782 non-null  object        
 8   New Subarea               415782 non-null  object        
 9   EE group                  415782 non-null  object        
 10  EESubgroup                415782 non-null  object        
 11  New organization          411305 non-null  object        
 12  Ne

In [135]:
action_list.columns



Index(['company code', 'Action Date', 'Action', 'Personnel No.', 'Name',
       'New Location', 'New Sector', 'Pers.area', 'New Subarea', 'EE group',
       'EESubgroup', 'New organization', 'New position', 'New job',
       'Old Subarea', 'Old location', 'Old Sector', 'Old organization',
       'Old position', 'Old job'],
      dtype='object')

In [ ]:
# Select the columns to merge from 'joblist'
columns_to_merge = ['Personnel No.', 'Level Sorting', 'Generation', 'Generation Sorting']

# Merge the selected columns from 'joblist' into 'basic data' based on the ''Concatenate - Position' column
action_list = action_list.merge(joblist[columns_to_merge], on='Personnel No.', how='left')

action_list.info()